In [1]:
import torch
from labels import id2label
from transformers import DistilBertForTokenClassification, AutoTokenizer
from datasets import load_from_disk

tokenizer = AutoTokenizer.from_pretrained("seelennebel/AM_tokenizer")    
model = DistilBertForTokenClassification.from_pretrained("seelennebel/AM")
dataset = load_from_disk("./dataset")

from sklearn.metrics import classification_report, precision_recall_fscore_support

c:\Users\seelennebel\dev\llm\llm-kernel\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Appending y_true with mbert_token_classes

In [2]:
y_true = []

for true_classes in dataset["test"]["mbert_token_classes"]:
    y_true.append(true_classes)

In [3]:
y_pred = []

for row in dataset["test"]:
    # converting tokens from mbert_tokens to ids
    ids = tokenizer.convert_tokens_to_ids(row["mbert_tokens"])
    # creating attention mask for all ids
    attention_mask = [1] * (len(ids) - 2)
    # deleting special token ids
    ids.pop(0)
    ids.pop(-1)

    inputs = {}
    inputs["input_ids"] = torch.tensor(ids).unsqueeze(0)
    inputs["attention_mask"] = torch.tensor(attention_mask).unsqueeze(0)

    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_labels = torch.argmax(probabilities, dim=-1)
    predicted_classes = []
    for value in predicted_labels[0]:
        predicted_classes.append(id2label[int(value)])

    y_pred.append(predicted_classes)

In [4]:
y_true_flat = [label for seq in y_true for label in seq]
y_pred_flat = [label for seq in y_pred for label in seq]
labels = list(id2label.values())

In [5]:
print(classification_report(y_true=y_true_flat, y_pred=y_pred_flat, labels=labels))

c:\Users\seelennebel\dev\llm\llm-kernel\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                    precision    recall  f1-score   support

                 O       0.94      1.00      0.97    111356
B-DRIVERLICENSENUM       0.88      0.07      0.13        99
I-DRIVERLICENSENUM       0.86      0.71      0.78       756
     B-DATEOFBIRTH       0.00      0.00      0.00       139
     I-DATEOFBIRTH       1.00      0.11      0.20       595
         B-ZIPCODE       1.00      0.01      0.01       145
         I-ZIPCODE       0.92      0.65      0.77       452
          B-TAXNUM       0.89      0.07      0.13       110
          I-TAXNUM       0.97      0.14      0.24       497
     B-BUILDINGNUM       0.00      0.00      0.00       155
    B-TELEPHONENUM       0.93      0.66      0.77       193
    I-TELEPHONENUM       0.84      0.94      0.88      1247
         B-SURNAME       1.00      0.01      0.02       336
         I-SURNAME       0.93      0.04      0.08       614
           B-EMAIL       0.99      0.87      0.93       219
           I-EMAIL       0.98      0.99

c:\Users\seelennebel\dev\llm\llm-kernel\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\seelennebel\dev\llm\llm-kernel\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
precision, recall, f1, _ = precision_recall_fscore_support(y_true_flat, y_pred_flat, labels=labels, average="weighted")
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Precision: 0.9296
Recall: 0.9358
F1 Score: 0.9142


c:\Users\seelennebel\dev\llm\llm-kernel\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
